In [1]:
import numpy as np
import polars as pl
import random

In [2]:
path_to_df = '../data/Clothing/data.csv'
df = pl.read_csv(
    path_to_df,
    has_header=False,
    new_columns=['user_id', 'item_id', 'rating', 'timestamp'],
    separator=',',
    schema_overrides={
        "user_id": pl.String,
        "item_id": pl.String,
        "rating": pl.Float64,
        "timestamp": pl.UInt64
    }
)
print(df.shape)

(5748920, 4)


In [3]:
df.head()

user_id,item_id,rating,timestamp
str,str,f64,u64
"""A2XVJBSRI3SWDI""","""0000031887""",5.0,1383523200
"""A2G0LNLN79Q6HR""","""0000031887""",4.0,1337990400
"""A2R3K1KX09QBYP""","""0000031887""",2.0,1361059200
"""A19PBP93OF896""","""0000031887""",1.0,1363824000
"""A1P0IHU93EF9ZK""","""0000031887""",4.0,1390435200


In [4]:
filtering_stage = 0
is_changed = True
threshold = 5
good_users = set()
good_items = set()

filtered_df = df.clone()

while is_changed:
    user_counts = filtered_df.group_by('user_id').agg(
        pl.len().alias('user_count'),
    )
    item_counts = filtered_df.group_by('item_id').agg(
        pl.len().alias('item_count'),
    )

    good_users = user_counts.filter(pl.col('user_count') >= threshold).select(
        'user_id',
    )
    good_items = item_counts.filter(pl.col('item_count') >= threshold).select(
        'item_id',
    )

    old_size = len(filtered_df)

    new_df = filtered_df.join(good_users, on='user_id', how='inner')
    new_df = new_df.join(good_items, on='item_id', how='inner')

    new_size = len(new_df)

    print(f'После {filtering_stage + 1}го этапа фильтрации.')
    print(f'Количество пользователей: {good_users.shape[0]}.')
    print(f'Количество фильмов: {good_items.shape[0]}')
    print()

    filtered_df = new_df
    is_changed = old_size != new_size
    filtering_stage += 1


filtered_df = filtered_df.with_columns(user_id=pl.col('user_id').rank('dense'))
filtered_df = filtered_df.with_columns(item_id=pl.col('item_id').rank('dense'))
filtered_df = filtered_df.sort(['user_id', 'timestamp'])

grouped_filtered_df = filtered_df.group_by('user_id', maintain_order=True).agg(
    pl.all().exclude('user_id'),
)

После 1го этапа фильтрации.
Количество пользователей: 185986.
Количество фильмов: 192978

После 2го этапа фильтрации.
Количество пользователей: 108489.
Количество фильмов: 59343

После 3го этапа фильтрации.
Количество пользователей: 68096.
Количество фильмов: 44710

После 4го этапа фильтрации.
Количество пользователей: 58834.
Количество фильмов: 33660

После 5го этапа фильтрации.
Количество пользователей: 49955.
Количество фильмов: 30521

После 6го этапа фильтрации.
Количество пользователей: 47067.
Количество фильмов: 27281

После 7го этапа фильтрации.
Количество пользователей: 43919.
Количество фильмов: 26154

После 8го этапа фильтрации.
Количество пользователей: 42759.
Количество фильмов: 24878

После 9го этапа фильтрации.
Количество пользователей: 41451.
Количество фильмов: 24410

После 10го этапа фильтрации.
Количество пользователей: 40946.
Количество фильмов: 23891

После 11го этапа фильтрации.
Количество пользователей: 40357.
Количество фильмов: 23676

После 12го этапа фильтрации

In [5]:
print('Users count:', filtered_df.select('user_id').unique().shape[0])
print('Items count:', filtered_df.select('item_id').unique().shape[0])
print('Actions count:', filtered_df.shape[0])
print(
    'Avg user history len:',
    np.mean(
        list(
            map(
                lambda x: x[0],
                grouped_filtered_df.select(
                    pl.col('item_id').list.len(),
                ).rows(),
            ),
        ),
    ),
)

Users count: 39387
Items count: 23033
Actions count: 278677
Avg user history len: 7.075354812501587


In [6]:
grouped_filtered_df = filtered_df.group_by("user_id", maintain_order=True).agg(
    pl.all().exclude("user_id")
)


In [7]:
grouped_filtered_df.head()

user_id,item_id,rating,timestamp
u32,list[u32],list[f64],list[u64]
1,"[9194, 4955, … 1884]","[5.0, 5.0, … 5.0]","[1350518400, 1350518400, … 1370908800]"
2,"[1904, 2148, … 20385]","[4.0, 4.0, … 3.0]","[1378684800, 1378684800, … 1384387200]"
3,"[16677, 15488, … 15504]","[3.0, 5.0, … 4.0]","[1388620800, 1388793600, … 1389225600]"
4,"[7505, 14432, … 21708]","[5.0, 5.0, … 5.0]","[1383868800, 1389139200, … 1405036800]"
5,"[11503, 8024, … 20468]","[5.0, 5.0, … 5.0]","[1369008000, 1371600000, … 1375920000]"


In [8]:
valid_portion = 0.1
test_portion = 0.1

all_user_ids = grouped_filtered_df.get_column("user_id").to_list()

random.seed(42)
random.shuffle(all_user_ids)

n_users = len(all_user_ids)
n_train = int(n_users * (1.0 - valid_portion - test_portion))
n_valid = int(n_users * valid_portion)

train_user_ids = set(all_user_ids[:n_train])
valid_user_ids = set(all_user_ids[n_train : n_train + n_valid])
test_user_ids = set(all_user_ids[n_train + n_valid:])

print(f"Users count: {n_users}")
print(f"Train users count: {len(train_user_ids)}")
print(f"Valid users count: {len(valid_user_ids)}")
print(f"Test users count: {len(test_user_ids)}")

Users count: 39387
Train users count: 31509
Valid users count: 3938
Test users count: 3940


In [9]:
train_samples = []
valid_samples = []
test_samples = []

max_len_train = 20

for user_id, item_history, _, _ in grouped_filtered_df.iter_rows():
    if user_id in train_user_ids:
        history = item_history[-max_len_train:]
        train_samples.append(
            {
                'user_id': user_id,
                'history': history,
            },
        )
    elif user_id in valid_user_ids:
        assert len(item_history) >= 5

        split_idx = int(len(item_history) * 0.8)
        assert not split_idx < 1 or split_idx >= len(item_history)
        
        history = item_history[:split_idx]
        target = item_history[split_idx:]

        valid_samples.append(
            {
                'user_id': user_id,
                'history': history,
                'target': target
            }
        )


    elif user_id in test_user_ids:
        assert len(item_history) >= 5

        split_idx = int(len(item_history) * 0.8)
        assert not split_idx < 1 or split_idx >= len(item_history)

        history = item_history[:split_idx]
        target = item_history[split_idx:]

        test_samples.append(
            {
                'user_id': user_id,
                'history': history,
                'target': target
            }
        )
    


In [10]:
len(train_samples), len(valid_samples), len(test_samples)

(31509, 3938, 3940)

In [11]:
# train
with open('../data/Clothing/train.txt', 'w') as f:
    for train_sample in train_samples:
        f.write(
            ' '.join(
                [str(train_sample['user_id'])]
                + [str(item_id) for item_id in train_sample['history']],
            ),
        )
        f.write('\n')

# valid
with open('../data/Clothing/valid_history.txt', 'w') as f_hist, open('../data/Clothing/valid_target.txt', 'w') as f_trg:
    for valid_sample in valid_samples:
        f_hist.write(' '.join([str(valid_sample['user_id'])] + [str(item_id) for item_id in valid_sample['history']]) + '\n')
        f_trg.write(' '.join([str(valid_sample['user_id'])] + [str(trg) for trg in valid_sample['target']]) + '\n')

# test
with open('../data/Clothing/test_history.txt', 'w') as f_hist, open('../data/Clothing/test_target.txt', 'w') as f_trg:
    for test_sample in test_samples:
        f_hist.write(' '.join([str(test_sample['user_id'])] + [str(item_id) for item_id in test_sample['history']]) + '\n')
        f_trg.write(' '.join([str(test_sample['user_id'])] + [str(trg) for trg in test_sample['target']]) + '\n')